In [ ]:
import pandas as pd
import pickle as pkl
import numpy as np
import rpy2.robjects as robjects
from collections import OrderedDict
import scipy.stats as stats
import scipy.linalg as linalg

In [ ]:
PKL_DATA_PATH = "/Users/raphaelkim/Dropbox (Harvard University)/HeartStepsV2V3/Raphael/all91.pkl"
PRIOR_DATA_PATH = "/Users/raphaelkim/Dropbox (Harvard University)/HeartStepsV2V3/Raphael/bandit-prior.RData"
NDAYS = 90
NUSERS = 91
NTIMES = 5

N_POST_SAMPLES = 5000

F_KEYS = ["intercept", "dosage", "engagement", "other_location", "variation"]
G_KEYS = ["intercept", "dosage", "engagement", "other_location", "variation", "temperature", "logpresteps", "sqrt_totalsteps"]

E0 = 0.2
E1 = 0.1


In [ ]:
# Load data
def load_data():
    with open(PKL_DATA_PATH, "rb") as f:
        data = pkl.load(f)
    return data

In [ ]:
def determine_user_state(data):
    '''Determine the state of each user at each time point'''
    availability = data[2]

    features = OrderedDict()
    features["intercept"] = 1
    features["dosage"] = data[6]
    features["engagement"] = data[7]
    features["other_location"] = data[8]
    # features["work_location"] = data[9]
    features["variation"] = data[10]
    features["temperature"] = data[11]
    features["logpresteps"] = data[12]
    features["sqrt_totalsteps"] = data[13]
    # features["prior_anti"] = data[14]

    fs = np.array([v for k,v in features.items() if k in F_KEYS])
    gs = np.array([v for k,v in features.items() if k in G_KEYS])

    return availability, fs, gs

In [ ]:
def load_priors():
    '''Load priors from RData file'''
    robjects.r['load'](PRIOR_DATA_PATH)
    priors = robjects.r['bandit.prior']
    alpha_pmean = np.array(priors.rx2("mu1"))
    alpha_psd = np.array(priors.rx2("Sigma1"))
    beta_pmean = np.array(priors.rx2("mu2"))
    beta_psd = np.array(priors.rx2("Sigma2"))
    sigma = float(priors.rx2("sigma")[0])

    prior_sigma = linalg.block_diag(alpha_psd, beta_psd, beta_psd)
    prior_mu = np.concatenate([alpha_pmean, beta_pmean, beta_pmean])

    return prior_sigma, prior_mu, sigma

In [ ]:
def get_priors_alpha_beta(post_mu, post_sigma):
    '''Get alpha and beta priors from mu and sigma'''
    alpha_pmean = post_mu[:len(G_KEYS)].flatten()
    alpha_psd = post_sigma[:len(G_KEYS), :len(G_KEYS)]
    beta_pmean = post_mu[-len(F_KEYS):].flatten()
    beta_psd = post_sigma[-len(F_KEYS):, -len(F_KEYS):]

    return alpha_pmean, alpha_psd, beta_pmean, beta_psd

In [ ]:
def sample_lr_params(alpha_pmean, alpha_psd, beta_pmean, beta_psd, sigma):
    '''Sample alpha, beta and noise from priors for BLR'''

    alpha0 = np.random.multivariate_normal(alpha_pmean, alpha_psd)
    alpha1 = np.random.multivariate_normal(beta_pmean, beta_psd)
    beta = np.random.multivariate_normal(beta_pmean, beta_psd)
    et = np.random.normal(0, np.sqrt(sigma**2))

    return alpha0, alpha1, beta, et

In [ ]:
def clip(x, eta = .2):#.2 is used in Peng's code
    '''Clipping function'''
    return min(1 - E0, max(x, E1))

In [ ]:
def posterior_draws_beta(post_mu, post_sigma, n=N_POST_SAMPLES):
    '''Draw n samples from beta's multivariate normal distribution'''
    _, _, beta_pmean, beta_psd = get_priors_alpha_beta(post_mu, post_sigma)
    return np.random.multivariate_normal(beta_pmean, beta_psd, n)

In [ ]:
def calculate_post_prob(fs, posterior_draws, eta = 0):
    '''Calculate the posterior probability of Pr(fs * b > eta)'''
    positive_samples = len(np.where(posterior_draws @ fs > eta)[0])
    post_prob = positive_samples / N_POST_SAMPLES
    phi_prob = clip(post_prob)
    return phi_prob

In [ ]:
def calculate_reward(post_mu, post_sigma, fs, gs, sigma, action, prob):

    # Get priors for alpha and beta
    alpha_pmean, alpha_psd, beta_pmean, beta_psd = get_priors_alpha_beta(post_mu, post_sigma)

    # Sample alpha, beta and noise
    alpha0, alpha1, beta, et = sample_lr_params(alpha_pmean, alpha_psd, beta_pmean, beta_psd, sigma)

    # Calculate reward
    reward = gs @ alpha0 + (prob * (fs @ alpha1)) + (action - prob) * (fs @ beta) + et

    return reward

In [ ]:
def calculate_phi(prob_matrix, action_matrix, fs_matrix, gs_matrix):
    Phi = np.expand_dims(np.hstack((gs_matrix, fs_matrix * prob_matrix.reshape(-1, 1), \
                (fs_matrix * (action_matrix - prob_matrix).reshape(-1, 1)))), axis=2)
    return Phi

In [ ]:
def calculate_post_sigma(prior_sigma, sigma, availability_matrix, Phi):
    '''Calculate the posterior sigma'''

    # Phi squared
    Phi_square = np.multiply(Phi, Phi.transpose(0, 2, 1))

    # Sum of availability times Phi squared
    avail_phi_squared_sum = np.sum(np.multiply(availability_matrix.reshape(-1, 1, 1), Phi_square), axis=0) / (sigma**2)

    # Posterior sigma
    post_sigma = np.linalg.inv(np.linalg.inv(prior_sigma) + avail_phi_squared_sum)

    return post_sigma

In [ ]:
def calculate_post_mu(prior_sigma, prior_mu, sigma, availability_matrix, reward_matrix, Phi, post_sigma):
    '''Calculate the posterior mu'''

    # Product of prior sigma inverse and prior mu
    sig_mu = (np.linalg.inv(prior_sigma) @ prior_mu.T).reshape(-1, 1)
    
    # Product of Phi and reward
    Phi_reward = np.multiply(Phi, reward_matrix.reshape(-1, 1, 1))

    # Sum of availability times Phi and reward
    avail_phi_reward_sum = np.sum(np.multiply(availability_matrix.reshape(-1, 1, 1), Phi_reward), axis=0)

    # Posterior mu
    post_mu = (post_sigma @ (sig_mu + (avail_phi_reward_sum/(sigma ** 2))) )

    return post_mu

In [ ]:
def calculate_posterior(prior_sigma, prior_mu, sigma, availability_matrix, prob_matrix, reward_matrix, action_matrix, fs_matrix, gs_matrix):
    '''Calculate the posterior distribution'''
    
    # Calculate phi(s, a)
    Phi = calculate_phi(prob_matrix, action_matrix, fs_matrix, gs_matrix)

    # Calculate posterior sigma
    post_sigma = calculate_post_sigma(prior_sigma, sigma, availability_matrix, Phi)

    # Calculate posterior mu
    post_mu = calculate_post_mu(prior_sigma, prior_mu, sigma, availability_matrix, reward_matrix, Phi, post_sigma)
    
    return post_mu[:,0], post_sigma

In [ ]:
def select_action(p):
    '''Select action from bernoulli distribution with probability p'''
    return stats.bernoulli.rvs(p)

In [ ]:
# Load data
data = load_data()

# Load priors
prior_sigma, prior_mu, sigma = load_priors()

# Posterior initialized using priors
post_sigma, post_mu = np.copy(prior_sigma), np.copy(prior_mu)

# DS to store availability, probabilities, features, actions and rewards
availability_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
prob_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
reward_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
action_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
fs_matrix = np.zeros((NUSERS, NDAYS * NTIMES, len(F_KEYS)))
gs_matrix = np.zeros((NUSERS, NDAYS * NTIMES, len(G_KEYS)))


for user in range(NUSERS):
# for user in range(80, 90):
    for day in range(NDAYS):

        # loop for each decision time during the day
        for time in range(NTIMES):

            # Get the current timeslot
            ts = (day) * 5 + time
            
            # State of the user at time ts
            availability, fs, gs = determine_user_state(data[user][ts])

            # Save user's availability
            availability_matrix[user, ts] = availability
            
            # If user is available
            if availability == 1:

                # Draw from the posterior
                beta_draws = posterior_draws_beta(post_mu, post_sigma)

                # Calculate probability of (fs x beta) > n
                #prob_fsb = calculate_post_prob(fs, beta_draws)
            
                
                # Sample action with probability prob_fsb from bernoulli distribution
                #action = select_action(prob_fsb)

                # Bayesian LR to estimate reward
                reward = calculate_reward(post_mu, post_sigma, fs, gs, sigma, action, prob_fsb)

                # Save probability, features, action and reward
                fs_matrix[user, ts] = fs
                gs_matrix[user, ts] = gs
                prob_matrix[user, ts] = prob_fsb
                action_matrix[user, ts] = action
                reward_matrix[user, ts] = reward
            
            # print(user, day, time, action_matrix[user, ts])

        # Update posterior
        post_mu, post_sigma = calculate_posterior(prior_sigma, prior_mu, sigma, availability_matrix[user][:ts + 1], prob_matrix[user][:ts + 1], 
                                                    reward_matrix[user][:ts + 1], action_matrix[user][:ts + 1], fs_matrix[user][:ts + 1], gs_matrix[user][:ts + 1])
        print(post_mu)

# Bootstrap

## A: Learning well on Average over time

Below, using calculate_reward (using the true rewards seen in the data leads to odd performance), while our sanity check calculate_reward_old (using reward generated from a linear contextual bandit with gaussian noise) leads to somewhat more reasonable plots. However, it is still a bit odd performance since posteriors are 0 in (both?) cases.
                    

In [ ]:
prior_sigma, prior_mu, sigma = load_priors()
print(prior_mu)
print(sigma)

In [ ]:
# since 0 posterior tx betas are learned, test for different rewards.
import random

random.seed(123)
true_mu= np.random.uniform(-1,1,(18,))
true_sigma= np.random.uniform(0,.1,(18,18))
true_sigma=np.matmul(np.transpose(true_sigma), true_sigma)

#print(true_sigma)
#print(true_mu)

In [ ]:
def determine_user_state(data):
    '''Determine the state of each user at each time point'''
    availability = data[2]

    features = OrderedDict()
    features["intercept"] = 1
    features["dosage"] = data[6]
    features["engagement"] = data[7]
    features["other_location"] = data[8]
    # features["work_location"] = data[9]
    features["variation"] = data[10]
    features["temperature"] = data[11]
    features["logpresteps"] = data[12]
    features["sqrt_totalsteps"] = data[13]
    # features["prior_anti"] = data[14]

    prob=data[3]
    action=data[4]
    
    reward = data[5]
    
    fs = np.array([v for k,v in features.items() if k in F_KEYS])
    gs = np.array([v for k,v in features.items() if k in G_KEYS])

    return availability, fs, gs, reward

In [ ]:
# calculate reward if not bootstrap through the linear model below
# o.w. 
def calculate_reward(post_mu, post_sigma, fs, gs, sigma, action, prob, reward, IS_BOOTSTRAP, bootstrap_metadata_user={}):
    if IS_BOOTSTRAP:
        alpha0 =  bootstrap_metadata_user['baseline_theta'][:len(G_KEYS)].flatten()
        alpha1 =  bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()
        beta=bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()   
        
        estimated_reward = gs @ alpha0 + (prob * (fs @ alpha1)) + (action - prob) * (fs @ beta)

        if bootstrap_metadata_user['useUser']: #user specific bootstrap
            reward = bootstrap_metadata_user['resampled_residuals_i'][0,bootstrap_metadata_user['ts']] + estimated_reward
        
        else: # population bootstrap
            reward= bootstrap_metadata_user['residuals'][bootstrap_metadata_user['ts']] + estimated_reward
    return reward

In [ ]:
#just for testing - CAN REMOVE
def calculate_reward_old(post_mu, post_sigma, fs, gs, sigma, action, prob, reward, IS_BOOTSTRAP, bootstrap_metadata_user={}):
    if IS_BOOTSTRAP:
        alpha0 =  bootstrap_metadata_user['baseline_theta'][:len(G_KEYS)].flatten()
        alpha1 =  bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()
        beta=bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()   
        
        estimated_reward = gs @ alpha0 + (prob * (fs @ alpha1)) + (action - prob) * (fs @ beta)

        if bootstrap_metadata_user['useUser']: #user specific bootstrap
            reward = bootstrap_metadata_user['resampled_residuals_i'][0,bootstrap_metadata_user['ts']] + estimated_reward
        else: # population bootstrap
            reward= bootstrap_metadata_user['residuals'][bootstrap_metadata_user['ts']] + estimated_reward
    else:
        # Get priors for alpha and beta
        alpha_pmean, alpha_psd, beta_pmean, beta_psd = get_priors_alpha_beta(post_mu, post_sigma)

        # Sample alpha, beta and noise
        alpha0, alpha1, beta, et = sample_lr_params(alpha_pmean, alpha_psd, beta_pmean, beta_psd, sigma)

        # Calculate reward
        reward = gs @ alpha0 + (prob * (fs @ alpha1)) + (action - prob) * (fs @ beta) + et
    return reward

In [ ]:
# Changes: 
# - functionize to return relevant data from a run, 
# - run individually on each user, 
# - as of now: the only change with bootstrap run vs regular run is (i) reward assignment and (ii) data used
#    - hence, we change the reward mechanism function accordingly
# - minor bug in not learning individually for each user, and updating posterior appropriately
# - dosage needs to be properly calculated

def run_algorithm(data, user_indices, IS_BOOTSTRAP, posterior_results={}):    
    # Load priors
    prior_sigma, prior_mu, sigma = load_priors()

    # DS to store availability, probabilities, features, actions and rewards
    availability_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    prob_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    reward_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    action_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    fs_matrix = np.zeros((NUSERS, NDAYS * NTIMES, len(F_KEYS)))
    gs_matrix = np.zeros((NUSERS, NDAYS * NTIMES, len(G_KEYS)))
    
    posteriors=[]

    for i in range(len(user_indices)):
        user=user_indices[i]
        # Posterior initialized using priors
        posteriors_i=[]
        post_sigma, post_mu = np.copy(prior_sigma), np.copy(prior_mu)
        bootstrap_metadata_user={'useUser': posterior_results['useUser'],'resampled_residuals_i': posterior_results['resampled_residuals_i'],'residuals': posterior_results['residuals'][user], 'baseline_theta': posterior_results['baseline_thetas'][user]} if IS_BOOTSTRAP else {}
        for day in range(NDAYS):
            for time in range(NTIMES):
                # Get the current timeslot
                ts = (day) * 5 + time

                # State of the user at time ts
                availability, fs, gs, reward = determine_user_state(data[user][ts])

                # Save user's availability
                availability_matrix[user, ts] = availability

                # If user is available
                if availability == 1:
                    # Draw from the posterior
                    beta_draws = posterior_draws_beta(post_mu, post_sigma)

                    # Calculate probability of (fs x beta) > n
                    prob_fsb = calculate_post_prob(fs, beta_draws)

                    # Sample action with probability prob_fsb from bernoulli distribution
                    action = select_action(prob_fsb)

                    # Bayesian LR to estimate reward
                    bootstrap_metadata_user['ts']= ts 
                    reward = calculate_reward(post_mu, post_sigma, fs, gs, sigma, action, prob_fsb, reward, IS_BOOTSTRAP, bootstrap_metadata_user)
                    #reward = calculate_reward_old(post_mu, post_sigma, fs, gs, sigma, action, prob_fsb, reward, IS_BOOTSTRAP, bootstrap_metadata_user)
                    #reward = calculate_reward_old(true_mu, true_sigma, fs, gs, sigma, action, prob_fsb, reward, IS_BOOTSTRAP, bootstrap_metadata_user)
                    
                    # Save probability, features, action and reward
                    fs_matrix[i, ts] = fs
                    gs_matrix[i, ts] = gs
                    prob_matrix[i, ts] = prob_fsb
                    action_matrix[i, ts] = action
                    reward_matrix[i, ts] = reward

            # Update posterior
            post_mu, post_sigma = calculate_posterior(prior_sigma, prior_mu, sigma, availability_matrix[i][:ts + 1], prob_matrix[i][:ts + 1], 
                                                        reward_matrix[i][:ts + 1], action_matrix[i][:ts + 1], fs_matrix[i][:ts + 1], gs_matrix[i][:ts + 1])
            posterior_it={'mu': post_mu, 'sigma': post_sigma}
            posteriors_i.append(posterior_it)
            #save post_mu, post_sigma for each person.
            
        posteriors.append(posteriors_i)

    result={'posteriors': posteriors, 'rewards': reward_matrix, 'fs_matrix':fs_matrix, 'gs_matrix': gs_matrix, 'action_matrix':action_matrix, 'prob_matrix': prob_matrix, 'availability_matrix':availability_matrix}
    result['useUser']=False
    result['resampled_residuals_i']={}
    result['resampled_residuals']={}
    return result


In [ ]:
# get data for bootstrap run. in particular:
# - add on \hat{\epsilon}_{it} to the result matrix, 
# - form baseline parameters with which to bootstrap (ie the prior). HERE is where we may specify alternative inputs of interest
def add_residual_pairs(result, baseline="Prior"):
    prior_sigma, prior_mu, sigma = load_priors()
    residual_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    baseline_thetas={}
    for user in range(NUSERS):
        posterior_user_T=result['posteriors'][user][NDAYS-1]
        for day in range(NDAYS):
            for time in range(NTIMES):
                ts = (day) * 5 + time
                # get estiamted reward,
                alpha, alpha_sd, beta, beta_sd = get_priors_alpha_beta(posterior_user_T['mu'], np.zeros(posterior_user_T['sigma'].shape))
                estimated_reward = result['gs_matrix'][user, ts] @ alpha + (result['prob_matrix'][user, ts] * (result['fs_matrix'][user, ts] @ beta)) + (result['action_matrix'][user, ts] - result['prob_matrix'][user, ts]) * (result['fs_matrix'][user, ts] @ beta)
                residual_matrix[user, ts]=result['rewards'][user][ts]-estimated_reward
        baseline_theta=posterior_user_T['mu']
        
        if baseline=="Prior":
            baseline_theta[-len(F_KEYS):]=prior_mu[-len(F_KEYS):]
        
        elif baseline=="Truth":#for testing
            baseline_theta[-len(F_KEYS):]=true_mu[-len(F_KEYS):] 
        
        baseline_thetas[user]=baseline_theta

    # 
    result['residuals']=residual_matrix
    result['baseline_thetas']=baseline_thetas
    return result


## Main Runner ##

In [ ]:
def determine_user_state(data):
    '''Determine the state of each user at each time point'''
    availability = data[2]

    features = OrderedDict()
    features["intercept"] = 1
    features["dosage"] = data[6]
    features["engagement"] = data[7]
    features["other_location"] = data[8]
    # features["work_location"] = data[9]
    features["variation"] = data[10]
    features["temperature"] = data[11]
    features["logpresteps"] = data[12]
    features["sqrt_totalsteps"] = data[13]
    # features["prior_anti"] = data[14]

    prob=data[3]
    action=data[4]
    reward = data[5]
    
    fs = np.array([v for k,v in features.items() if k in F_KEYS])
    gs = np.array([v for k,v in features.items() if k in G_KEYS])

    return availability, fs, gs, reward,prob, action

In [ ]:
import pickle
np.random.seed(0)

def run_algorithm(data, user_indices, IS_BOOTSTRAP, posterior_results={}):    
    # Load priors
    prior_sigma, prior_mu, sigma = load_priors()

    # DS to store availability, probabilities, features, actions and rewards
    availability_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    prob_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    reward_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    action_matrix = np.zeros((NUSERS, NDAYS * NTIMES))
    fs_matrix = np.zeros((NUSERS, NDAYS * NTIMES, len(F_KEYS)))
    gs_matrix = np.zeros((NUSERS, NDAYS * NTIMES, len(G_KEYS)))
    
    posteriors=[]

    for i in range(len(user_indices)):
        nanHit=False
        user=user_indices[i]
        # Posterior initialized using priors
        posteriors_i=[]
        post_sigma, post_mu = np.copy(prior_sigma), np.copy(prior_mu)
        
        #bootstrap_metadata_user={'useUser': posterior_results['useUser'],'resampled_residuals_i': posterior_results['resampled_residuals_i'],'residuals': posterior_results['residuals'][user], 'baseline_theta': posterior_results['baseline_thetas'][user]} if IS_BOOTSTRAP else {}
        for day in range(NDAYS):
            for time in range(NTIMES):
                # Get the current timeslot
                ts = (day) * 5 + time

                # State of the user at time ts
                availability, fs, gs, reward,p_data,a_data = determine_user_state(data[user][ts])

                # Save user's availability
                availability_matrix[user, ts] = availability

                # If user is available
                if availability == 1:
                    # Draw from the posterior
                    ## beta_draws = posterior_draws_beta(post_mu, post_sigma)

                    # Calculate probability of (fs x beta) > n
                    prob_fsb = calculate_post_prob(fs, beta_draws)

                    # Sample action with probability prob_fsb from bernoulli distribution
                    action = select_action(prob_fsb)

                    # Bayesian LR to estimate reward
                    #bootstrap_metadata_user['ts']= ts 
                    #reward = calculate_reward(post_mu, post_sigma, fs, gs, sigma, action, prob_fsb, reward, IS_BOOTSTRAP, bootstrap_metadata_user)
                    #reward = calculate_reward_old(post_mu, post_sigma, fs, gs, sigma, action, prob_fsb, reward, IS_BOOTSTRAP, bootstrap_metadata_user)
                    #reward = calculate_reward_old(true_mu, true_sigma, fs, gs, sigma, action, prob_fsb, reward, IS_BOOTSTRAP, bootstrap_metadata_user)
                    
                    # Save probability, features, action and reward
                    prob_matrix[i, ts] = p_data#prob_fsb
                    action_matrix[i, ts] = a_data#action
                    reward_matrix[i, ts] = reward
                
                fs_matrix[i, ts] = fs
                gs_matrix[i, ts] = gs

            #print("FS "+str(fs_matrix))
            #print("GS "+str(gs_matrix))
            #print("Prob "+str(prob_matrix))
            #print("Action "+str(action_matrix))
            #print("Reward "+str(reward_matrix))
            
            # Update posterior
            post_mu, post_sigma = calculate_posterior(prior_sigma, prior_mu, sigma, availability_matrix[i][:ts + 1], prob_matrix[i][:ts + 1], 
                                                        reward_matrix[i][:ts + 1], action_matrix[i][:ts + 1], fs_matrix[i][:ts + 1], gs_matrix[i][:ts + 1])
            
            if np.sum(np.isnan(post_mu))>0 and not nanHit:
                print("User: "+str(i)+". Time: "+str(time)+", ts: "+str(ts)) 
                nanHit=True
            posterior_it={'mu': post_mu, 'sigma': post_sigma}
            posteriors_i.append(posterior_it)
            #save post_mu, post_sigma for each person.
            
        posteriors.append(posteriors_i)
        
    result={'posteriors': posteriors, 'rewards': reward_matrix, 'fs_matrix':fs_matrix, 'gs_matrix': gs_matrix, 'action_matrix':action_matrix, 'prob_matrix': prob_matrix, 'availability_matrix':availability_matrix}
    result['useUser']=False
    result['resampled_residuals_i']={}
    result['resampled_residuals']={}
    return result

def initial_run():
    data = load_data()
    result=run_algorithm(data, [10], IS_BOOTSTRAP=False) #range(NUSERS), IS_BOOTSTRAP=False)
    #with open('/Users/raphaelkim/Dropbox (Harvard University)/HeartStepsV2V3/Raphael/original_result_91.pickle', 'wb') as handle:
    #    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return result

result=initial_run()

In [ ]:
#print(result['rewards'][0])
data[10,0:10,[2, 5]]
user=10
timeStart=0
timeEnd=35
cols=[2,3,4,14,6]

cols=[2,5]
print(pd.DataFrame(data[user, 0:50])[cols])

#print(sum(np.isnan(pd.DataFrame(data[user, 0:1350][5]))))

#pd.DataFrame(data[0:91, 0:1350][5])


In [ ]:
print(data[user][0:1350])

# Main Runner

In [ ]:
data = load_data()
result=run_algorithm(data, range(NUSERS), IS_BOOTSTRAP=False)

baseline="Prior" #"Posterior","Truth"
result=add_residual_pairs(result, baseline=baseline)
result['useUser']=False

B=2 # Number of bootstrap simulations
bootstrapped_results={}
for b in range(B):
    resampled_indices = np.random.choice(range(NUSERS), NUSERS)
    result_b=run_algorithm(data, resampled_indices, True, result)
    bootstrapped_results[b]=result_b

bs_avg_a = bootstrapped_results

Why is there nan and 0000 posterior T effect when recalculating posterior from the og algorithm (Prasidh's data)??

In [ ]:
# pd.DataFrame(data[0, 380:420])[[3, 4, 14, 6]]
for i in range(NUSERS):
    #print(pd.DataFrame(data[i, :]).columns.values)
    index_i=pd.DataFrame(data[i, :])[[2]].ne(0).idxmax()
    
    print(index_i)
    
    prob=pd.DataFrame(data[0, index_i:(index_i+1)])[[2,3,4,14,6]]
    print(prob)

In [ ]:
print(pd.DataFrame(data[0, 285:290])[[2,3,4,14,6]])

print(pd.DataFrame(data[1, 0:35])[[2,3,4,14,6]])

print(pd.DataFrame(data[2, 210:220])[[2,3,4,14,6]])



In [ ]:
print(pd.DataFrame(data[82, 0:35])[[2,3,4,14,6]])

# Cleaner Analysis for Learning on Average

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd
import pickle as pkl
import numpy as np
import rpy2.robjects as robjects
from collections import OrderedDict
import scipy.stats as stats
import scipy.linalg as linalg
import argparse
import os

# %%
PKL_DATA_PATH = "/Users/raphaelkim/Dropbox (Harvard University)/HeartStepsV2V3/Raphael/all91.pkl"
PRIOR_DATA_PATH = "/Users/raphaelkim/Dropbox (Harvard University)/HeartStepsV2V3/Raphael/bandit-prior.RData"
NDAYS = 90
NUSERS = 91
NTIMES = 5

LAMBDA = 0.95

F_KEYS = ["intercept", "dosage", "engagement", "other_location", "variation"]
F_LEN = len(F_KEYS)
G_KEYS = ["intercept", "dosage", "engagement", "other_location", "variation", "temperature", "logpresteps", "sqrt_totalsteps"]
G_LEN = len(G_KEYS)

#"outputDir/results_{user}_{boot_num}.pkl"



    

In [ ]:
#result = {"availability": availability_matrix, "prob": prob_matrix, "action": action_matrix, "reward": reward_matrix,
#              "fs": fs_matrix, "gs": gs_matrix, "post_mu": post_mu_matrix, "post_sigma": post_sigma_matrix}

## B: User Specific Bootstrap

In [ ]:
# now we do a user specific bootstrap: Each bootstrap trajectory is generated similarly as in the population bootstrap 
# with the only difference that the residual added to the reward at an available time point
# is sampled with replacement from all of the residuals from this same user.
def resample_user_residuals(result, i, baseline="Prior"):
    T= NDAYS * NTIMES
    residual_matrix_resampled = np.zeros((1, T))
    resampled_indices = np.random.choice(range(T), T)
    residual_matrix_resampled[0,:]=result['residuals'][i][resampled_indices]
    result['resampled_residuals_i']=residual_matrix_resampled
    return result

In [ ]:
B=10 # Number of bootstrap simulations
bootstrapped_results={}
result['useUser']=True
i=15

for b in range(B):
    result=resample_user_residuals(result, i) #using this
    result_b=run_algorithm(data, [i], True, result)
    bootstrapped_results[b]=result_b

In [ ]:
%matplotlib inline
plotResult(bootstrapped_results, result, state, key="posteriors", baseline="Posterior", user=i)

## Preliminary code for calculating interestingness according to the definition in JMIR draft
I was initially thinking that we just calculate the difference of avergae (conditional) treatment effects in engaged vs not state.

In [ ]:
def calculateInterestingness(result, x=0, y=35, delta1=0.463, delta2=1, key="posteriors"):
    standardizedEffects={}
    engaged={}
    for user in range(len(result[key])):#this will always be 1 for a user specific bootstrap
        standardizedEffectsUser=[]
        engagedUser=[]
        for day in range(NDAYS):
            for time in range(NTIMES):
                ts = (day) * 5 + (NTIMES-1)

                beta=result['posteriors'][user][day]['mu'][-len(F_KEYS):]
                mean =result['fs_matrix'][user, ts] @ beta

                sigma=result['posteriors'][user][day]['sigma'][-len(F_KEYS):, -len(F_KEYS):]
                std=(result['fs_matrix'][user, ts] @ sigma) @ result['fs_matrix'][user, ts]
                
                standardizedEffectsUser.append(mean/std)
                engagedUser.append(result['fs_matrix'][user, ts][2]) #check engagement is at index 2!!!
        standardizedEffects[user]=np.array(standardizedEffectsUser)
        engaged[user]=np.array(engagedUser)
    result['standardizedEffects']=standardizedEffects
    result['engaged']=engaged

    # iterate through sliding windows
    n=(y-1)//2
    iterations=range(len(result[key]))
    total=len(iterations)
    statistics={}
    for user in iterations:
        statistic={}
        tIndex=0
        nUndeterminedSlidingWindows=0
        nInterestingDeterminedWindows=0
        nSlidingWindows=0
        nDeterminedWindows=0
        determinedTimes=[]
        unDeterminedTimes=[]
        for day in range(NDAYS):
            for time in range(NTIMES):
                ts = (day) * 5 + (NTIMES-1)
                if result['availability_matrix'][user, ts]: 
                    nSlidingWindows=nSlidingWindows+1
                    
                    effects=[]
                    engages=[]

                    if ts <= n: 
                        effects=result['standardizedEffects'][user][0:ts+n]
                        engages=result['engaged'][user][1:ts+n]
                    elif ts > n and ts <= NDAYS*NTIMES-1-n:
                        effects=result['standardizedEffects'][user][(ts-n):(ts+n)]
                        engages=result['engaged'][user][(ts-n):(ts+n)]
                    else:
                        effects=result['standardizedEffects'][user][(ts-n):(NDAYS*NTIMES-1)]
                        engages=result['engaged'][user][(ts-n):(NDAYS*NTIMES-1)]
                    isUnderdetermined = (sum(engages) >=x) and (len(engages)-sum(engages) >= x)
                    if not isUnderdetermined:
                        nDeterminedWindows=nDeterminedWindows+1
                        determinedTimes.append(ts)

                        # claculate effects
                        engageIndices=np.where(engages==1)[0]
                        nonEngageIndices=np.where(engages==0)[0]
                        avgEngage=np.mean(effects[engageIndices])
                        avgNonEngage=np.mean(effects[nonEngageIndices])
                        if avgEngage > avgNonEngage:
                            nInterestingDeterminedWindows=nInterestingDeterminedWindows+1
                    else:
                        nUndeterminedSlidingWindows=nUndeterminedSlidingWindows+1
                        unDeterminedTimes.append(ts)
        if nSlidingWindows >0 and nDeterminedWindows >0:
            statistic["r1"]=nUndeterminedSlidingWindows/nSlidingWindows
            statistic["r2"]=nInterestingDeterminedWindows/nDeterminedWindows
            statistic["isInteresting"]=(statistic["r1"]<delta1) and (abs(statistic["r2"]-.5)>delta2)
        else: 
            statistic["r1"]=statistic["r2"]=statistic["isInteresting"]=None
        statistic["determinedTimes"]=determinedTimes
        statistic["unDeterminedTimes"]=unDeterminedTimes

        statistic["engaged"]=result['engaged'][user]
        statistic["standardizedEffects"]=result['standardizedEffects'][user]
        statistics[user]=statistic
    return statistics

In [ ]:
interestingnessOriginal=calculateInterestingness(result)
bsAvgInteresting=[]
bsUserInteresting=[]
for b in range(len(bs_avg_a)):
    bsAvgInteresting.append(calculateInterestingness(bs_avg_a[b]))

for b in range(len(bootstrapped_results)):
    bsUserInteresting.append(calculateInterestingness(bootstrapped_results[b]))

In [ ]:
import matplotlib.pyplot as plt

def plotEngagementResult(result, user):
    determinedTimes=result[user]['determinedTimes']
    unDeterminedTimes=result[user]['unDeterminedTimes']
    
    bsThickness=1.5
    opacity=1
    plt.clf()
    f = plt.figure()
    f.set_figwidth(20)
    f.set_figheight(10)
    plt.plot(determinedTimes, result[user]['standardizedEffects'][determinedTimes], color='b', linewidth=bsThickness, alpha=opacity)
    plt.plot(unDeterminedTimes, result[user]['standardizedEffects'][unDeterminedTimes], color='r',label="Bootstrapped Posterior Means", linewidth=bsThickness, alpha=opacity)

    plt.legend(loc="upper right")
    plt.title('Stdized Posterior Tx Effect for engaged and not')
    plt.ylabel('Stdized Posterior Tx Effect')
    plt.xlabel('Decision Time') 
    plt.show()
    

In [ ]:
plotEngagementResult(interestingnessOriginal, 0)

In [ ]:
#Random checks.

t=np.array([1,0,1])
indices=np.where(t==1)[0]

f=np.array([11,10,11])
np.mean(f[indices])

print(result.keys())

last=bootstrapped_results[0]['posteriors'][0][0]['mu']
lastOg=result['posteriors'][i][0]['mu']

for t in range(90):
    v1=(np.linalg.norm(result['posteriors'][i][0]['mu']-lastOg))
    v2=(np.linalg.norm(bootstrapped_results[0]['posteriors'][0][t]['mu']-last))
    #print("Og: "+str(v1)+"\t\t Bs: "+str(v2))
    last=bootstrapped_results[0]['posteriors'][0][t]['mu']
    lastOg=result['posteriors'][i][t]['mu']

# Exploring curve summaries

- calculate probabiliteis of curves
- calculate probabilities of engage effect > not engage effect (given available)

In [ ]:
#interestingnessOriginal
#result
from scipy.stats import multivariate_normal

# aggregate results
def getProbabilities(result, search="interesting"):
    probMetric={}
    for user in range(NUSERS):
        tIndex=0
        probs=[]
        for day in range(NDAYS):
            for time in range(NTIMES):
                ts = (day) * 5 + (NTIMES-1)
                val=0
                if search=="interesting":
                    #alpha0 = result['posteriors'][user][day]['mu'][:len(G_KEYS)]
                    #alpha1 = result['posteriors'][user][day]['mu'][-len(F_KEYS):]

                    #alpha0 =  bootstrap_metadata_user['baseline_theta'][:len(G_KEYS)].flatten()
                    #alpha1 =  bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()
                    #beta=bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()   
                    
                    #alpha0 =  bootstrap_metadata_user['baseline_theta'][:len(G_KEYS)].flatten()
                    #alpha1 =  bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()
                    #beta=bootstrap_metadata_user['baseline_theta'][-len(F_KEYS):].flatten()   
        
                    #estimated_reward = gs @ alpha0 + (prob * (fs @ alpha1)) + (action - prob) * (fs @ beta)
                    beta = result['posteriors'][user][day]['mu'][-len(F_KEYS):]
                    beta_psd = result['posteriors'][user][day]['sigma'][-len(F_KEYS):, -len(F_KEYS):]

                    curFS=result['fs_matrix'][user, ts]
                    curFS=curFS.reshape(curFS.shape[0],1)
                    curFS=np.transpose(curFS)                

                    curFS[0,2]=1
                    mean1=np.matmul(curFS, beta)
                    var1=np.matmul(curFS, beta_psd)
                    var1=np.matmul(var1, np.transpose(curFS))                    
                    
                    curFS[0,2]=0
                    mean2=np.matmul(curFS, beta)
                    var2=np.matmul(curFS, beta_psd)
                    var2=np.matmul(var2, np.transpose(curFS))                    
                    #dist2=multivariate_normal(mean2, var2)  
                    
                    dist1Minus2=multivariate_normal(mean1-mean2, var1+var2)
                    probs.append(dist1Minus2.cdf(0))
                    # get mvn
                #get p(tau()-tau()>0)
                # get P(beta > beta_b)
        probMetric[user]=probs
        result['probMetricInteresting']=probMetric
    return result



In [ ]:
def aggregate_time(result):
    # aggregrate over time
    vals=[]
    for day in range(NDAYS):
        for time in range(NTIMES):
            val_t=0
            for i in range(NUSERS):
                ts = (day) * 5 + (NTIMES-1)
                val_t=val_t+result['probMetricInteresting'][i][ts]
            val_t=val_t/NUSERS
            vals.append(val_t)
    result['observedAvgProbMetricInteresting']=vals
    return result

result=getProbabilities(result)
result=aggregate_time(result)
true_p=sum(result['observedAvgProbMetricInteresting'])/len(result['observedAvgProbMetricInteresting'])

In [ ]:
allBootstrappedPs=[]
for b in range(B):
    bootstrapped_results[b]=getProbabilities(bootstrapped_results[b])
    bootstrapped_results[b]=aggregate_time(bootstrapped_results[b])
    res_p=bootstrapped_results[b]['observedAvgProbMetricInteresting']
    p_b=sum(res_p)/len(res_p)
    allBootstrappedPs.append(p_b)

In [ ]:
from scipy.stats import percentileofscore
true_p=sum(result['observedAvgProbMetricInteresting'])/len(result['observedAvgProbMetricInteresting'])
print(true_p)
print(allBootstrappedPs)
percentileofscore(allBootstrappedPs, true_p)

# prob curves greater

In [ ]:
print(result['posteriors'][0][0]['mu'][-len(F_KEYS):].shape)
print(type(result['posteriors'][0][0]['mu'][-len(F_KEYS):]))

In [ ]:
print(result['posteriors'][0][0]['sigma'][-len(F_KEYS):, -len(F_KEYS):])
state=np.array([1, 3.86, 1, 1, 0]) # following the target state in JMIR Draft 04-01-21
state=state.reshape(state.shape[0],1)
print(state.shape)

In [ ]:
#interestingnessOriginal
#result
from scipy.stats import multivariate_normal

state=np.array([1, 3.86, 1, 1, 0]) # following the target state in JMIR Draft 04-01-21
state=state.reshape(state.shape[0],1)
state=np.transpose(state)

def computeDistributions(result):
    distributionsByDay=[]
    beta=np.zeros(result['posteriors'][0][0]['mu'][-len(F_KEYS):].shape)
    beta_psd=np.zeros(result['posteriors'][0][0]['sigma'][-len(F_KEYS):, -len(F_KEYS):].shape)
    for day in range(NDAYS):
        for user in range(NUSERS):
            beta = beta+ result['posteriors'][user][day]['mu'][-len(F_KEYS):]
            beta_psd = beta_psd + result['posteriors'][user][day]['sigma'][-len(F_KEYS):, -len(F_KEYS):]
        beta=beta/NUSERS
        beta_psd=beta_psd/(NUSERS**2)
        distribution_day={'mu': beta, 'sigma': beta_psd}
        distributionsByDay.append(distribution_day)
    return distributionsByDay

# aggregate results
def getProbabilitiesCurves(result, bootstrapped_results):
    probs=[]
    observed_distros=computeDistributions(result)
    for b in range(B):
        bootstrapped_distros=computeDistributions(bootstrapped_results[b])
        for day in range(NDAYS):
            mean=observed_distros[day]['mu']-bootstrapped_distros[day]['mu']
            var=observed_distros[day]['sigma']+bootstrapped_distros[day]['sigma']
            mean=np.matmul(state, mean)
            var=np.matmul(np.matmul(state, var), np.transpose(state))
            distDifference=multivariate_normal(mean, var)
            #distDifference=multivariate_normal(observed_distros[day]['mu']-bootstrapped_distros[day]['mu'], observed_distros[day]['sigma']+bootstrapped_distros[day]['sigma'])
            probs.append(distDifference.cdf(0))
    result['bootstrapped_comparison_blue_over_black']=probs #list that p(blue > black_b) for each b, across all time points
    return result

result=getProbabilitiesCurves(result, bootstrapped_results)


In [ ]:
print(result['bootstrapped_comparison_blue_over_black'])

In [ ]:
print(len(result['posteriors'][0])) #[user][ts]

In [ ]:
print("hi")

In [ ]:
print(F_KEYS)

In [ ]:
print(G_KEYS)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

u=[1,2,3,4]
m=[0,-.75,2,-.5]
l=[-4,-3,-2,-1]
labels=["one", "two", "three", "four"]
z=1.96
color='#2187bb'
horizontal_line_width=0.25
dat={'u': u, 'm':m,'l':l, 'labels': labels}

df=pd.DataFrame(dat)
df=df.sort_values(by=['m'])
remap={}
for i in range(df.shape[0]):
    remap[df.index.values[i]]=i
df=df.rename(index = remap)
for i in range(df.shape[0]):
    left = i - horizontal_line_width / 2
    top = df['u'][i]
    right = i + horizontal_line_width / 2
    bottom = df['l'][i]
    label=df['labels'][i]
    mean=df['m'][i]
    
    plt.plot([top, bottom], [label,label], color=color)
    plt.plot([top, top], [left, right], color=color)
    plt.plot([bottom, bottom],[left, right], color=color)
    plt.plot(mean,i, 'o', color='#f44336')
    
